In [2]:
import pandas as pd
import json
import re

In [ ]:
import pandas as pd
import json
import re
import unicodedata

# Fonction de normalisation des adresses avec gestion des valeurs manquantes
def normalize_address(address):
    if not isinstance(address, str):  # Vérifie si ce n'est pas une chaîne
        return ""  # Retourne une chaîne vide pour éviter l'erreur
    address = re.sub(r'/', '-', address)  # Remplace les / par -
    address = unicodedata.normalize('NFD', address).encode('ascii', 'ignore').decode("utf-8")  # Supprime les accents
    return re.sub(r'[^a-zA-Z0-9 .\-()]', '', address).lower().strip()

# Charger les fichiers JSON en DataFrames
with open("fichier_1.json", "r", encoding="utf-8") as file1:
    df1 = pd.DataFrame(json.load(file1))

with open("fichier_2.json", "r", encoding="utf-8") as file2:
    df2 = pd.DataFrame(json.load(file2))

# Appliquer la normalisation aux colonnes concernées
df1['normalized_la_street'] = df1['la_street'].apply(normalize_address)
df2['normalized_or_street'] = df2['or_street'].apply(normalize_address)

# Trouver les lignes où la_street de df1 correspond à or_street de df2
matching_rows = df1[df1['normalized_la_street'].isin(df2['normalized_or_street'])]

# Afficher les résultats
print(matching_rows)

# Optionnel : Enregistrer les résultats dans un fichier JSON
matching_rows.to_json("matching_addresses.json", orient="records", force_ascii=False, indent=4)

print("Les correspondances ont été enregistrées dans 'matching_addresses.json'")


In [19]:
import unicodedata
# Fonction de normalisation des adresses avec gestion des valeurs manquantes
# Fonction de normalisation des adresses avec gestion des valeurs manquantes
def normalize_address(address):
    if not isinstance(address, str):  # Vérifie si ce n'est pas une chaîne
        return ""  # Retourne une chaîne vide pour éviter l'erreur
    address = re.sub(r'/', '-', address)  # Remplace les / par -
    address = unicodedata.normalize('NFD', address).encode('ascii', 'ignore').decode("utf-8")  # Supprime les accents
    return re.sub(r'[^a-zA-Z0-9 .\-()]', '', address).lower().strip()

# Charger les fichiers JSON en DataFrames
with open("../logistic_address_20250213_084740.json", "r", encoding="utf-8") as file1:
    df1 = pd.DataFrame(json.load(file1))

with open("../organization_20250213_084740.json", "r", encoding="utf-8") as file2:
    df2 = pd.DataFrame(json.load(file2))

# Appliquer la normalisation aux colonnes concernées
df1['la_street'] = df1['la_street'].apply(normalize_address)
df2['or_street'] = df2['or_street'].apply(normalize_address)

In [36]:
df2.sample()

,or_id,or_denomination,or_rna,or_house_number,or_street,or_postal_code,or_city,or_country,or_state,or_logo,...,fk_us,fk_ot,fk_ovs,fk_cou,fileFi_id,categories,logistic_address,origin_approvals,positioning,organization_categories
33,33,ANTI GASPI SOLIDAIRE,W841302378,86,avenue de saint jean 84130,84,LE PONTET,,,None,...,None,None,None,None,None,[],[],[],[],[]


In [37]:
# Fonction pour extraire et compléter les codes postaux
def extract_postal_code(postal_code, street):
    match_five_digits = re.search(r'\b(\d{5})\b', street)
    match_two_digits = re.search(r'\b(\d{2})\b', postal_code)
    
    if re.match(r'\b\d{5}\b', postal_code):
        return postal_code  # Si déjà un code postal à 5 chiffres, on le garde
    
    if match_two_digits and match_five_digits:
        first_two = match_five_digits.group(1)[:2]  # Deux premiers chiffres du code postal à 5 chiffres
        if match_two_digits.group(1) == first_two:
            return match_five_digits.group(1)  # Associe le code postal si les deux chiffres correspondent
        else:
            return None  # Si la correspondance échoue, on retournera ce cas dans un autre JSON
    
    return ""  # Si aucun code postal valide trouvé, on met une chaîne vide

# Charger les fichiers JSON en DataFrames
with open("../logistic_address_20250213_084740.json", "r", encoding="utf-8") as file1:
    df1 = pd.DataFrame(json.load(file1))

with open("../organization_20250213_084740.json", "r", encoding="utf-8") as file2:
    df2 = pd.DataFrame(json.load(file2))

# Complétion des codes postaux
df1['filled_la_postal_code'] = df1.apply(lambda row: extract_postal_code(str(row['la_postal_code']), str(row['la_street'])), axis=1)
df2['filled_or_postal_code'] = df2.apply(lambda row: extract_postal_code(str(row['or_postal_code']), str(row['or_street'])), axis=1)

# Enregistrer les cas où la correspondance des codes postaux échoue
invalid_postal_codes = df1[df1['filled_la_postal_code'].isnull()]
invalid_postal_codes.to_json("invalid_postal_codes.json", orient="records", force_ascii=False, indent=4)

print("Les codes postaux non correspondants ont été enregistrés dans 'invalid_postal_codes.json'")


Les codes postaux non correspondants ont été enregistrés dans 'invalid_postal_codes.json'
